# OpenAI Assistant V2 - RAG 테스트

- 저작자: [테디노트](https://www.youtube.com/@teddynote)

**코드 재배포시 출처를 남겨 주세요**

In [1]:
# 라이브러리 업데이트
# !pip install -qU langchain-teddynote

**OpenAI API KEY 발급 방법**

- [OpenAI 키 발급방법](https://wikidocs.net/233342)

In [2]:
# 발급 받은 키를 입력해 주세요
openai_api_key = "OPENAI API KEY 입력"

`.env` 에 OpenAI API KEY 를 저장해 두었다면 밑의 코드 주석을 해제하고 실행하세요.

In [3]:
from dotenv import load_dotenv
import os

load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")

## 주요 링크

- [OpenAI Assistant 대시보드](https://platform.openai.com/assistants)
- [OpenAI API KEY 발급](https://platform.openai.com/api-keys)
- [OpenAI Files 스토리지](https://platform.openai.com/storage/files)

## 실습에 활용한 문서

소프트웨어정책연구소(SPRi) - 2023년 12월호

- 저자: 유재흥(AI정책연구실 책임연구원), 이지수(AI정책연구실 위촉연구원)
- 링크: https://spri.kr/posts/view/23669
- 파일명: `SPRI_AI_Brief_2023년12월호_F.pdf`

## STEP 1) 설정

- `_DEFAULT_RAG_INSTRUCTIONS` 는 RAG 의 instruction 입니다. 프롬프트를 자유롭게 수정하여 테스트해 보세요.
- `OPENAI_API_KEY` 는 발급받은 OPENAI API KEY 를 입력해 주세요.
- `PROJECT_NAME` 은 프로젝트 이름을 입력해 주세요.
- `model_name` 은 사용할 모델을 입력해 주세요. (예, `gpt-4o`, `gpt-4o-mini`, ...)
- `chunk_size` 는 청크 크기를 입력해 주세요.
- `chunk_overlap` 는 청크 중복 크기를 입력해 주세요.

In [4]:
from langchain_teddynote.models import OpenAIAssistant


# RAG 시스템 프롬프트 입력
_DEFAULT_RAG_INSTRUCTIONS = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Answer in Korean."""


# 설정(configs)
configs = {
    "OPENAI_API_KEY": openai_api_key,  # OpenAI API 키
    "instructions": _DEFAULT_RAG_INSTRUCTIONS,  # RAG 시스템 프롬프트
    "PROJECT_NAME": "PDF-RAG-TEST",  # 프로젝트 이름(자유롭게 설정)
    "model_name": "gpt-4o",  # 사용할 OpenAI 모델 이름(gpt-4o, gpt-4o-mini, ...)
    "chunk_size": 1000,  # 청크 크기
    "chunk_overlap": 100,  # 청크 중복 크기
}


# 인스턴스 생성
assistant = OpenAIAssistant(configs)

## STEP 2) 파일 업로드

### 파일 업로드 하지 않은 경우

- RAG 할 파일을 업로드 합니다.
- 파일 업로드는 딱 1번만 하면 됩니다.

이미 파일을 업로드 하였고, file_id 가 있다면, 아래 코드는 생략합니다.

In [5]:
# 업로드할 파일 경로
data = "SPRI_AI_Brief_2023년12월호_F.pdf"

# 파일 업로드 후 file_id 는 잘 보관해 두세요. (대시보드에서 나중에 확인 가능)
file_id = assistant.upload_file(data)

## STEP 3-1) Assistant, VectorStore 설정

(참고) 이미 있는 assistant_id, vector_id 가 있다면, `STEP 3-2` 으로 넘어가세요.

- `assistant_id`, `vector_id` 는 잘 보관해 두세요 (메모장 같은 곳에...)

In [6]:
# 업로드한 파일의 ID 리스트 생성
file_ids = [file_id]

# 새로운 어시스턴트 생성 및 ID 받기
assistant_id, vector_id = assistant.create_new_assistant(file_ids)

# 어시스턴트 설정
assistant.setup_assistant(assistant_id)

# 벡터 스토어 설정
assistant.setup_vectorstore(vector_id)

## STEP 3-2) 이미 있는 assistant_id, vector_id 가 있다면, 아래와 같이 설정합니다.

만약에 이미 있는 assistant_id, vector_id 가 있다면, 아래와 같이 설정합니다.

In [7]:
# assistant_id = "asst_TdJbouMO7gU........."
# vector_id = "vs_ibSoSSErFDnS9........"

# # 어시스턴트 설정
# assistant.setup_assistant(assistant_id)

# # 벡터 스토어 설정
# assistant.setup_vectorstore(vector_id)

## STEP 4) 대화

- `stream()` 스트리밍 기능을 지원합니다.
- `invoke()` 는 일반 출력입니다. 답변 생성이 끝날때까지 기다렸다가 한 번에 출력합니다.

`stream()` 스트리밍 기능을 지원합니다.

In [8]:
for token in assistant.stream("삼성전자가 개발한 생성형 AI의 이름은?"):
    print(token, end="", flush=True)

삼성전자가 개발한 생성형 AI의 이름은 '삼성 가우스'입니다【4:0†SPRI_AI_Brief_2023년12월호_F.pdf】.

아래는 좀 더 편리하게 스트리밍 출력을 해줍니다. (단, 직접 토큰을 받아서 화면에 뿌려야 하는 경우에는 이전 코드가 더 유용합니다.)

In [9]:
from langchain_teddynote.messages import stream_response

stream_response(assistant.stream("이전 답변을 영어로"))

The name of the generative AI developed by Samsung is 'Samsung GAUSS'【4:0†SPRI_AI_Brief_2023년12월호_F.pdf】.

`invoke()` 는 일반 출력입니다. 답변 생성이 끝날때까지 기다렸다가 한 번에 출력합니다.

In [10]:
# 질문
print(assistant.invoke("삼성전자가 개발한 생성형 AI의 이름은?"))

삼성전자가 개발한 생성형 AI의 이름은 '삼성 가우스'입니다【4:0†SPRI_AI_Brief_2023년12월호_F.pdf】.


In [11]:
# 추가 질문
print(assistant.invoke("이전의 답변에 대한 출처(페이지)를 알려줘"))

이전 답변의 출처는 문서의 21페이지입니다【4:0†SPRI_AI_Brief_2023년12월호_F.pdf】.


대화 목록을 조회합니다.

In [12]:
# 대화 목록 조회
assistant.list_chat_history()

[{'content': '삼성전자가 개발한 생성형 AI의 이름은?', 'role': 'user'},
 {'content': "삼성전자가 개발한 생성형 AI의 이름은 '삼성 가우스'입니다【4:0†SPRI_AI_Brief_2023년12월호_F.pdf】.",
  'role': 'assistant'},
 {'content': '이전 답변을 영어로', 'role': 'user'},
 {'content': "The name of the generative AI developed by Samsung is 'Samsung GAUSS'【4:0†SPRI_AI_Brief_2023년12월호_F.pdf】.",
  'role': 'assistant'},
 {'content': '삼성전자가 개발한 생성형 AI의 이름은?', 'role': 'user'},
 {'content': "삼성전자가 개발한 생성형 AI의 이름은 '삼성 가우스'입니다【4:0†SPRI_AI_Brief_2023년12월호_F.pdf】.",
  'role': 'assistant'},
 {'content': '이전의 답변에 대한 출처(페이지)를 알려줘', 'role': 'user'},
 {'content': '이전 답변의 출처는 문서의 21페이지입니다【4:0†SPRI_AI_Brief_2023년12월호_F.pdf】.',
  'role': 'assistant'}]

새로운 주제는 새로운 대화 쓰레드(thread)를 시작해야 합니다.

In [13]:
# 대화 초기화
assistant.clear_chat_history()

새로운 주제의 대화를 시작합니다.

In [14]:
# 질문
stream_response(
    assistant.stream("구글이 앤쓰로픽에 투자 계획을 발표했습니다. 이에 대해 알려줘")
)

구글은 앤쓰로픽에 최대 20억 달러를 투자하기로 합의했으며, 먼저 5억 달러를 투자하고 향후 15억 달러를 추가로 투자할 계획입니다. 또한, 앤쓰로픽은 구글의 클라우드 서비스를 사용하기 위해 4년간 30억 달러 규모의 계약을 체결했습니다【4:0†SPRI_AI_Brief_2023년12월호_F.pdf】.

In [15]:
# 추가 질문
stream_response(assistant.stream("이전의 답변을 보고서 양식으로 작성해 주세요"))

# 투자 보고서: 구글의 앤쓰로픽 투자 계획

## 투자 개요
- **투자자**: 구글
- **투자 대상**: 앤쓰로픽
- **총 투자 금액**: 최대 20억 달러

## 투자 일정 및 세부사항
- **1차 투자**: 5억 달러
- **추가 투자 계획**: 15억 달러

## 계약 세부사항
- **클라우드 서비스 계약**: 앤쓰로픽은 구글과 클라우드 서비스를 사용하기 위한 4년간 30억 달러 규모의 계약을 체결

## 결론
이번 투자는 구글이 앤쓰로픽과의 협력 관계를 강화하고, 클라우드 서비스 분야에서 경쟁력을 확대할 수 있는 전략적 투자로 분석됩니다. 

In [16]:
# 대화 목록 조회
assistant.list_chat_history()

[{'content': '구글이 앤쓰로픽에 투자 계획을 발표했습니다. 이에 대해 알려줘', 'role': 'user'},
 {'content': '구글은 앤쓰로픽에 최대 20억 달러를 투자하기로 합의했으며, 먼저 5억 달러를 투자하고 향후 15억 달러를 추가로 투자할 계획입니다. 또한, 앤쓰로픽은 구글의 클라우드 서비스를 사용하기 위해 4년간 30억 달러 규모의 계약을 체결했습니다【4:0†SPRI_AI_Brief_2023년12월호_F.pdf】.',
  'role': 'assistant'},
 {'content': '이전의 답변을 보고서 양식으로 작성해 주세요', 'role': 'user'},
 {'content': '# 투자 보고서: 구글의 앤쓰로픽 투자 계획\n\n## 투자 개요\n- **투자자**: 구글\n- **투자 대상**: 앤쓰로픽\n- **총 투자 금액**: 최대 20억 달러\n\n## 투자 일정 및 세부사항\n- **1차 투자**: 5억 달러\n- **추가 투자 계획**: 15억 달러\n\n## 계약 세부사항\n- **클라우드 서비스 계약**: 앤쓰로픽은 구글과 클라우드 서비스를 사용하기 위한 4년간 30억 달러 규모의 계약을 체결\n\n## 결론\n이번 투자는 구글이 앤쓰로픽과의 협력 관계를 강화하고, 클라우드 서비스 분야에서 경쟁력을 확대할 수 있는 전략적 투자로 분석됩니다. ',
  'role': 'assistant'}]